# Jonathan Halverson
# Tuesday, February 7, 2017
# Gensim: Corpora and vector spaces

In [1]:
>>> import logging
>>> logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
>>> from gensim import corpora
>>>
>>> documents = ["Human machine interface for lab abc computer applications",
>>>              "A survey of user opinion of computer system response time",
>>>              "The EPS user interface management system",
>>>              "System and human system engineering testing of EPS",
>>>              "Relation of user perceived response time to error measurement",
>>>              "The generation of random binary unordered trees",
>>>              "The intersection graph of paths in trees",
>>>              "Graph minors IV Widths of trees and well quasi ordering",
>>>              "Graph minors A survey"]

2017-02-08 19:15:30,620 : INFO : 'pattern' package not found; tag filters are not available for English


In [3]:
>>> # remove common words and tokenize
>>> stoplist = set('for a of the and to in'.split())
>>> texts = [[word for word in document.lower().split() if word not in stoplist] for document in documents]

In [4]:
>>> # remove words that appear only once in the corpus
>>> from collections import defaultdict
>>> frequency = defaultdict(int)
>>> for text in texts:
>>>     for token in text:
>>>         frequency[token] += 1

>>> texts = [[token for token in text if frequency[token] > 1] for text in texts]

In [5]:
>>> from pprint import pprint  # pretty-printer
>>> pprint(texts)

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]


In [6]:
# alternative method to remove words that appear only once in the corpus
from collections import Counter
c = Counter(reduce(lambda u, v: u + v, texts))
texts = [[token for token in text if c[token] > 1] for text in texts]

In [7]:
>>> from pprint import pprint  # pretty-printer
>>> pprint(texts)

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]


In [8]:
>>> dictionary = corpora.Dictionary(texts)
>>> print(dictionary)

2017-02-08 19:15:30,674 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2017-02-08 19:15:30,675 : INFO : built Dictionary(12 unique tokens: [u'minors', u'graph', u'system', u'trees', u'eps']...) from 9 documents (total 29 corpus positions)


Dictionary(12 unique tokens: [u'minors', u'graph', u'system', u'trees', u'eps']...)


In [9]:
>>> print(dictionary.token2id)

{u'minors': 11, u'graph': 10, u'system': 6, u'trees': 9, u'eps': 8, u'computer': 1, u'survey': 5, u'user': 7, u'human': 2, u'time': 4, u'interface': 0, u'response': 3}


In [10]:
>>> new_doc = "Human computer interaction"
>>> new_vec = dictionary.doc2bow(new_doc.lower().split())
>>> print(new_vec)  # the word "interaction" does not appear in the dictionary and is ignored

[(1, 1), (2, 1)]


In [11]:
>>> corpus = [dictionary.doc2bow(text, return_missing=False) for text in texts]
>>> pprint(corpus)

[[(0, 1), (1, 1), (2, 1)],
 [(1, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)],
 [(0, 1), (6, 1), (7, 1), (8, 1)],
 [(2, 1), (6, 2), (8, 1)],
 [(3, 1), (4, 1), (7, 1)],
 [(9, 1)],
 [(9, 1), (10, 1)],
 [(9, 1), (10, 1), (11, 1)],
 [(5, 1), (10, 1), (11, 1)]]


One can write a class which overrides the "__iter__" function to generate one document at a time.


There are ways to serialize a corpus using different formats. The reading and writing is done in a streaming fashing meaning one document at a time. This makes it possible to work with large corpora within getting into problems with memory limitations. A dictionary of the vocabulary can also be constructed without loads all the documents (see the six module). 

# Topics and transformations

In this tutorial, I will show how to transform documents from one vector representation into another. This process serves two goals:

     
-- To bring out hidden structure in the corpus, discover relationships between words and use them to describe the documents in a new and (hopefully) more semantic way.

-- To make the document representation more compact. This both improves efficiency (new representation consumes less resources) and efficacy (marginal data trends are ignored, noise-reduction).

In [12]:
dictionary.items()

[(11, u'minors'),
 (10, u'graph'),
 (6, u'system'),
 (9, u'trees'),
 (8, u'eps'),
 (1, u'computer'),
 (5, u'survey'),
 (7, u'user'),
 (2, u'human'),
 (4, u'time'),
 (0, u'interface'),
 (3, u'response')]

In [13]:
corpus

[[(0, 1), (1, 1), (2, 1)],
 [(1, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)],
 [(0, 1), (6, 1), (7, 1), (8, 1)],
 [(2, 1), (6, 2), (8, 1)],
 [(3, 1), (4, 1), (7, 1)],
 [(9, 1)],
 [(9, 1), (10, 1)],
 [(9, 1), (10, 1), (11, 1)],
 [(5, 1), (10, 1), (11, 1)]]

In [14]:
from gensim import models, similarities

In [15]:
>>> tfidf = models.TfidfModel(corpus) # step 1 -- initialize a model

2017-02-08 19:15:30,746 : INFO : collecting document frequencies
2017-02-08 19:15:30,747 : INFO : PROGRESS: processing document #0
2017-02-08 19:15:30,748 : INFO : calculating IDF weights for 9 documents and 11 features (28 matrix non-zeros)


From now on, tfidf is treated as a read-only object that can be used to convert any vector from the old representation (bag-of-words integer counts) to the new representation (TfIdf real-valued weights):

In [16]:
>>> doc_bow = [(0, 1), (1, 1)]
>>> print(tfidf[doc_bow]) # step 2 -- use the model to transform vectors

[(0, 0.7071067811865476), (1, 0.7071067811865476)]


In [17]:
>>> corpus_tfidf = tfidf[corpus]
>>> for doc in corpus_tfidf:
...     print(doc)

[(0, 0.5773502691896257), (1, 0.5773502691896257), (2, 0.5773502691896257)]
[(1, 0.44424552527467476), (3, 0.44424552527467476), (4, 0.44424552527467476), (5, 0.44424552527467476), (6, 0.3244870206138555), (7, 0.3244870206138555)]
[(0, 0.5710059809418182), (6, 0.4170757362022777), (7, 0.4170757362022777), (8, 0.5710059809418182)]
[(2, 0.49182558987264147), (6, 0.7184811607083769), (8, 0.49182558987264147)]
[(3, 0.6282580468670046), (4, 0.6282580468670046), (7, 0.45889394536615247)]
[(9, 1.0)]
[(9, 0.7071067811865475), (10, 0.7071067811865475)]
[(9, 0.5080429008916749), (10, 0.5080429008916749), (11, 0.695546419520037)]
[(5, 0.6282580468670046), (10, 0.45889394536615247), (11, 0.6282580468670046)]


In [18]:
>>> lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=2) # initialize an LSI transformation
>>> corpus_lsi = lsi[corpus_tfidf] # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi

2017-02-08 19:15:30,770 : INFO : using serial LSI version on this node
2017-02-08 19:15:30,771 : INFO : updating model with new documents
2017-02-08 19:15:30,772 : INFO : preparing a new chunk of documents
2017-02-08 19:15:30,773 : INFO : using 100 extra samples and 2 power iterations
2017-02-08 19:15:30,774 : INFO : 1st phase: constructing (12, 102) action matrix
2017-02-08 19:15:30,775 : INFO : orthonormalizing (12, 102) action matrix
2017-02-08 19:15:30,777 : INFO : 2nd phase: running dense svd on (12, 9) matrix
2017-02-08 19:15:30,780 : INFO : computing the final decomposition
2017-02-08 19:15:30,781 : INFO : keeping 2 factors (discarding 47.565% of energy spectrum)
2017-02-08 19:15:30,782 : INFO : processed documents up to #9
2017-02-08 19:15:30,784 : INFO : topic #0(1.594): 0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"response" + 0.060*"time" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"
2017-02-08 19:15:30,785 : INFO : topic #

Here we transformed our Tf-Idf corpus via Latent Semantic Indexing into a latent 2-D space (2-D because we set num_topics=2). Now you’re probably wondering: what do these two latent dimensions stand for? Let’s inspect with models.LsiModel.print_topics():

In [19]:
>>> lsi.print_topics(2)

2017-02-08 19:15:30,792 : INFO : topic #0(1.594): 0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"response" + 0.060*"time" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"
2017-02-08 19:15:30,794 : INFO : topic #1(1.476): 0.460*"system" + 0.373*"user" + 0.332*"eps" + 0.328*"interface" + 0.320*"time" + 0.320*"response" + 0.293*"computer" + 0.280*"human" + 0.171*"survey" + -0.161*"trees"


[(0,
  u'0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"response" + 0.060*"time" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"'),
 (1,
  u'0.460*"system" + 0.373*"user" + 0.332*"eps" + 0.328*"interface" + 0.320*"time" + 0.320*"response" + 0.293*"computer" + 0.280*"human" + 0.171*"survey" + -0.161*"trees"')]

It appears that according to LSI, “trees”, “graph” and “minors” are all related words (and contribute the most to the direction of the first topic), while the second topic practically concerns itself with all the other words. As expected, the first five documents are more strongly related to the second topic while the remaining four documents to the first topic:

In [20]:
>>> for doc in corpus_lsi: # both bow->tfidf and tfidf->lsi transformations are actually executed here, on the fly
...     print(doc)
[(0, -0.066), (1, 0.520)] # "Human machine interface for lab abc computer applications"
[(0, -0.197), (1, 0.761)] # "A survey of user opinion of computer system response time"
[(0, -0.090), (1, 0.724)] # "The EPS user interface management system"
[(0, -0.076), (1, 0.632)] # "System and human system engineering testing of EPS"
[(0, -0.102), (1, 0.574)] # "Relation of user perceived response time to error measurement"
[(0, -0.703), (1, -0.161)] # "The generation of random binary unordered trees"
[(0, -0.877), (1, -0.168)] # "The intersection graph of paths in trees"
[(0, -0.910), (1, -0.141)] # "Graph minors IV Widths of trees and well quasi ordering"
[(0, -0.617), (1, 0.054)] # "Graph minors A survey"

[(0, 0.066007833960903012), (1, 0.52007033063618546)]
[(0, 0.19667592859142469), (1, 0.76095631677000541)]
[(0, 0.089926399724462758), (1, 0.72418606267525143)]
[(0, 0.075858476521780169), (1, 0.63205515860034289)]
[(0, 0.10150299184980158), (1, 0.57373084830029608)]
[(0, 0.70321089393783098), (1, -0.16115180214025687)]
[(0, 0.87747876731198304), (1, -0.16758906864659276)]
[(0, 0.90986246868185772), (1, -0.14086553628718873)]
[(0, 0.61658253505692795), (1, 0.053929075663894696)]


[(0, -0.617), (1, 0.054)]